# 02 RAG | 01 Chat Completion | 06 Chat Completion Image

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create ChatClient

The `IChatClient` from the Microsoft.Extensions.AI NuGet package offers an abstraction which allows using models from multiple providers.

The multi modal capabilities of the GPT-4o LLM deployment are shown by providing the following photo realistic image together with the user message 'In which style is the provided image? Is it Cartoon or Photorealistic?' the LLM:

![PhotoRealisticImage](../../Assets/Image/SuperBowl_Photorealistic.png)

The image was created using the [04_CreateImageCompletion notebook](./04_CreateImageCompletion.ipynb) and an instance of dall-e-3. 


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3"
#r "nuget: Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3"
#r "nuget: DotNetEnv, 3.1.1"

using Azure;
using Azure.AI.Inference;
using Microsoft.Extensions.AI;
using DotNetEnv;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";    
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";
oAiEndpoint = $"{oAiEndpoint}openai/deployments/{chatCompletionDeploymentName}";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
        
IChatClient chatClient =
    new ChatCompletionsClient(new Uri(oAiEndpoint), azureKeyCredential)
        .AsChatClient(chatCompletionDeploymentName);

Console.WriteLine($"IChatClient created...");

Installed Packages DotNetEnv, 3.1.1 Microsoft.Extensions.AI, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.Abstractions, 9.3.0-preview.1.25161.3 Microsoft.Extensions.AI.AzureAIInference, 9.3.0-preview.1.25161.3

IChatClient created...


## Step 1: Create Chat Messages

The following cell, demonstrate a basic interaction using the `chatClient` object. In this case, the system message is used to provide instructions or settings for the assistant. It instructs the model to *'Describe the style of images!'*.

The system message is converted into a `ChatMessage` and added to a `List<ChatMessage>` which will finally provided to the model.

The image file content is retrieved as `byte[]` array and added as `DataContent` to a `List<AIContent>` list. The user message to detect the style of the imagge is provided as `TextContent`to the `List<AIContent>`. Both input data is added to the `List<ChatMessages>` which already contains the system message. 

In [6]:
using System.IO;

string systemMessageRaw = "You describe the style of images!";
string userMessageRaw = "In which style is the provided image? Is it Cartoon or Photorealistic?";
string imageFileName = "../../Assets/Image/SuperBowl_Photorealistic.png"; 

List<ChatMessage> chatMessages = new List<ChatMessage>
{
    new ChatMessage(Microsoft.Extensions.AI.ChatRole.System, systemMessageRaw)
};

TextContent systemMessage = new TextContent(systemMessageRaw);
TextContent userMessage = new TextContent(userMessageRaw);

byte[] imageBytes = File.ReadAllBytes(imageFileName);
DataContent imageContent = new DataContent(imageBytes, "image/jpeg");

List<AIContent> aiContents = new List<AIContent>{
    userMessage,
    imageContent
};
chatMessages.Add(new ChatMessage(Microsoft.Extensions.AI.ChatRole.User, aiContents));

Console.WriteLine($"ChatMessages created...");

ChatMessages created...


## Step 2: Call Model

The method `GetResponseAsync()`of the `ChatClient` object can be used to provide the data to the model and retrieve the model response. 

In [7]:
ChatResponse chatResponse = await chatClient.GetResponseAsync(chatMessages);

Console.WriteLine($"Model response: {chatResponse.Text}");


Model response: The image is in a photorealistic style.
